In [ ]:
from os import environ
environ['PYGAME_HIDE_SUPPORT_PROMPT'] = '1'

import pygame  # it is important to import pygame after that
import time
import os
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# descomentar estas 4 próximas linhas para usar o navegador invisível
#gecko = '/home/andre/Downloads/geckodriver'
#driver = webdriver.FirefoxOptions()
#driver.add_argument('-headless')
#driver = webdriver.Firefox(executable_path=gecko, options=driver)

def abre_jogos():

    html = driver.execute_script("return document.documentElement.outerHTML")
    page = bs(html, 'html.parser')

    # variável "tam" vai receber a quantidade de jogo ao vivo
    tam = ''

    # contando o número de jogos ao vivo
    jogos_vivo = driver.find_element_by_css_selector('.js-sport-games-live-count > span:nth-child(2)').text
    tam1 = len(jogos_vivo)
       
    # eliminando os parênteses na busca do número de jogos ao vivo
    for apenas_numero in range(tam1):
        if (jogos_vivo[apenas_numero] != '(') and (jogos_vivo[apenas_numero] != ')'):
            tam = tam + jogos_vivo[apenas_numero]

    tam = int(tam)                                                 
    print(f'número de jogos ao vivo {tam}')
  
    partida = False
    
    tam_jogos = len(page.find('',{'class':'event-list-table-wrapper js-event-list-table-wrapper'})\
                    .find_all('',{'class':'js-event-list-tournament-events'}))

    # Percorrendo todos os jogos da rodada
    num_aba = 0
    cont_jogo = 0
    for j in range(tam_jogos):
        
        # contando quantos jogos tem na classe "js-event-list-tournament-events"
        tam_jogos_final = len(page.find_all('',{'class':'js-event-list-tournament-events'})[j].find_all('a'))
        
        for i in range(tam_jogos_final):
            cont_jogo += 1
            try: 

                # clicando nos jogos e abrindo em nova aba
                #html = driver.execute_script("return document.documentElement.outerHTML")
                #page = bs(html, 'html.parser')
            
                for tente_outra_vez in range(4):
                    try:                
                        pagina = page.find_all('',{'class':'js-event-list-tournament-events'})[j].find_all('a')[i].get('href')
                        pagina = 'https://www.sofascore.com' + pagina 

                        # abre a nova aba
                        driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 't')
                        print(f'abri o jogo {cont_jogo}')
                        
                        # muda de aba   
                        driver.switch_to.window(driver.window_handles[num_aba + 1])
                        num_aba += 1
                        break
                    except:
                        driver.refresh()
                        time.sleep(7)
                        print(f'exceção para abrir aba do jogo {cont_jogo}')
                        continue    
                
                # acessa nova página
                driver.get(pagina)
                time.sleep(7)                                

                # verificando se o jogo possui estatísticas                                              
                teste = driver.find_element_by_css_selector('.statistics-container > h3:nth-child(1)').text
                partida = True
                
            except:
                      
                print(f'exceção de estatistica do jogo {cont_jogo}')

                # fecha a aba
                driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 'w')
                excecao = True

                # mudar para a aba aberta antes - página principal 
                driver.switch_to.window(driver.window_handles[0])
                
                # atualiza o número de páginas abertas pois uma foi fechada
                num_aba -= 1

                # pula para o próximo jogo
                continue        
                
    return num_aba, partida


def scanner():
    
    possebola = 'Posse de bola'
    chutes = 'Finalizações'
    chutesnogol = 'Finalizações no gol'
    chutesforagol = 'Finalizações para fora'
    chutesbloqueados = 'Chutes travados'
    corner = 'Escanteios'
    impedimentos = 'Impedimentos'
    faltas = 'Faltas'
    cartoesa = 'Cartões amarelos'
    cartoesv = 'Cartões vermelhos'
    grandeschances = 'Grandes chances'
    grandeschancesperdidas = 'Grandes chances perdidas'
    chutesnatrave = 'Finalizações na trave'
    contraataques = 'Contra-ataques'
    chutesdentroarea = 'Finalizações de dentro da área'
    chutesforaarea = 'Finalizações de fora da área'
    defesasgoleiro = 'Defesas do goleiro'
    passes = 'Passes'
    passescertos = 'Passes certos'
    bolaslongas = 'Bolas longas'
    cruza = 'Cruzamentos'
    dribles = 'Dribles'
    posseperdida = 'Possession lost'
    duelosganhos = 'Duelos ganhos'
    disputasaereasvencidas = 'Disputas aéreas vencidas'
    desarmes = 'Desarmes'
    intercept = 'Interceptações'
    cortes = 'Cortes'
    
    mingol = []
    resultadopormin = []
    resultadomin = []
    goltime = []
    golxtime1min = []
    golxtime2min = []
        
    # mudar para a aba aberta antes - página principal 
    driver.switch_to.window(driver.window_handles[i + 1])

    # Coletando informações do jogo                            

    pais = driver.find_element_by_css_selector('.js-breadcrumb-category-link').text
    campeonato = driver.find_element_by_css_selector('.js-breadcrumb-tournament-link').text
    jogo = driver.find_element_by_css_selector('.js-event-page-event-name').text

    minuto_atual = driver.find_element_by_css_selector('.js-event-status-description').text
    resultado_parcial = driver.find_element_by_css_selector('.h1').text
    resultado_parcial = resultado_parcial[0] + 'x' + resultado_parcial[4]                 

    # rolar a página mais para baixo
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1) 

    html = driver.execute_script("return document.documentElement.outerHTML")
    page = bs(html, 'html.parser')


    # aqui começa a coleta de minuto de 1o gol 2o gol, placar em 5 e 5 min, quem fez o 1o
    # o 2o gol do jogo              

    # contando quantos gols aconteceram no jogo
    tam_placar = len(page.find_all('',{'class':'incident__goal'}))

    # contando quantos gols cada time fez
    tam_minutogol = len(page.find('div',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--incident '}))
    tam_minutogol2 = len(page.find('div',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--right cell--incident '}))

    # coletando o minuto dos gols

    # minuto que o placar mudou e novo placar
    for totalgols in range(tam_placar):
        placarpormin = page.find_all('',{'class':'incident__goal'})[totalgols].text.strip().replace(' - ','x')
        resultadopormin.append(placarpormin)

        # pegando na posição 0 o placar e na próxima o minuto e assim por diante
        # o minuto ele pega no trecho abaixo
        resultadomin.append(placarpormin)


    # gols do time 1
    for minutosgols in range(tam_minutogol):
        try:
            teste = page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--incident '})[minutosgols].find('span',{'class':'incident__goal'}).text.strip().replace(' - ','x')
            minutogol = int(page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--incident '})[minutosgols].find('',{'cell__content incident__time'}).text.strip().split("'")[0])
            mingol.append(minutogol)                                                                                                                         

            # verificando os gols depois dos 90 min
            if mingol[len(mingol) - 1] == 90:
                try:                                                                                                                                                                                                      
                    mingolacrescimo = page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--incident '})[minutosgols].find('',{'cell__content incident__time'}).find('sup').text.strip().split("+")[1] 
                    mingolacrescimo = int(mingolacrescimo) + int(mingol[len(mingol) - 1])
                    mingol[len(mingol) - 1] = mingolacrescimo
                except:
                    pass

            resultadomin.append(mingol[len(mingol) - 1])

            # goltime armazena o placar e quem fez o gol
            goltime.append(teste)
            goltime.append('1')

            # verifica o minuto do 1o, 2o, 3o gol do time 1
            golxtime1min.append(mingol[len(mingol) - 1])

        except:
            continue

    # gols do time 2       
    for minutosgols2 in range(tam_minutogol2):
        try:
            teste2 = page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--right cell--incident '})[minutosgols2].find('span',{'class':'incident__goal'}).text.strip().replace(' - ','x')
            minutogol = int(page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--right cell--incident '})[minutosgols2].find('',{'cell__content incident__time'}).text.strip().split("'")[0])
            mingol.append(minutogol)

            # verificando os gols depois dos 90 min
            if mingol[len(mingol) - 1] == 90:
                try:                                                                                                                                                                                                      
                    mingolacrescimo = page.find('',{'class':'js-event-page-incidents-container'}).find_all('',{'class':'cell cell--right cell--incident '})[minutosgols2].find('',{'cell__content incident__time'}).find('sup').text.strip().split("+")[1] 
                    mingolacrescimo = int(mingolacrescimo) + int(mingol[len(mingol) - 1])
                    mingol[len(mingol) - 1] = mingolacrescimo
                except:
                    pass


            resultadomin.append(mingol[len(mingol) - 1])

            # goltime armazena o placar e quem fez o gol
            goltime.append(teste2)
            goltime.append('2')

            # verifica o minuto do 1o, 2o, 3o gol do time 2
            golxtime2min.append(mingol[len(mingol) - 1])

        except:
            continue   

    # vamos precisar saber o tamanho desse array
    tam_goltime = len(goltime)

    # variáveis auxiliares que receberam o placar por min de gol e o min dos gols sendo ordenadas
    resultadopormin.reverse()
    golxtime1min.reverse()
    golxtime2min.reverse()
    mingol.sort()

    # testando se o jogo foi 0x0
    if resultado_parcial == '0x0':
        mingol.append('-1')
        resultadopormin.append('0x0')
        goltime.append('null')
        resultadomin.append('0x0')
        resultadomin.append('-1')

      
    # quantidade de dados 
    tam_blocos = len(page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})) 
    
    # scaneando os dados
    for tam_blocos1 in range(tam_blocos):
        try:
          
            nome_dado = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find('',{'class':'cell__section--main u-tC'}).text
            
            if nome_dado.strip() == possebola:

                possebolacasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                possebolavisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()
 
            
            elif nome_dado.strip() == chutes:

                chutescasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                chutesvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == chutesnogol:

                chutesnogolcasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                chutesnogolvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == chutesforagol:

                chutesforagolcasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                chutesforagolvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == chutesbloqueados:

                chutesbloqueadoscasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                chutesbloqueadosvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == corner: 

                cornercasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                cornervisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == impedimentos:

                impedimentoscasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                impedimentosvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == faltas:

                faltascasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                faltasvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == cartoesa:

                cartoesacasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                cartoesavisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == cartoesv:

                cartoesvcasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                cartoesvvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == grandeschances:

                grandeschancescasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                grandeschancesvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == grandeschancesperdidas:

                grandeschancesperdidascasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                grandeschancesperdidasvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == chutesnatrave:

                chutesnatravecasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                chutesnatravevisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == contraataques:

                contraataquescasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                contraataquesvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == chutesdentroarea:

                chutesdentroareacasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                chutesdentroareavisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == chutesforaarea:

                chutesforaareacasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                chutesforaareavisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == defesasgoleiro:

                defesasgoleirocasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                defesasgoleirovisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == passes:

                passescasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                passesvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == passescertos:

                passescertoscasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                passescertosvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == bolaslongas:

                bolaslongascasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                bolaslongasvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == cruza:

                cruzacasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                cruzavisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == dribles:

                driblescasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                driblesvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == posseperdida:

                posseperdidacasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                posseperdidavisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == duelosganhos:

                duelosganhoscasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                duelosganhosvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == disputasaereasvencidas:

                disputasaereasvencidascasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                disputasaereasvencidasvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == desarmes:

                desarmescasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                desarmesvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == intercept:

                interceptcasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                interceptvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()

            elif nome_dado.strip() == cortes:

                cortescasa = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[0].text.strip()
                cortesvisitante = page.find(id = 'statistics-period-ALL').find_all('',{'class':'cell cell--incident'})[tam_blocos1].find_all('',{'class':'cell__section'})[1].text.strip()            
        
        except:
            continue

    print('\n')
    print(pais)                             
    print(campeonato)
    print(jogo)
    print(f'Resultado Parcial {resultado_parcial}')
    print(f'Minuto {minuto_atual}')
    print(f'Chutes no gol do time da casa {chutesnogolcasa}')
    print(f'Chutes no gol do time visitante {chutesnogolvisitante}')   
            
    # MÉTODO BACK
    if minuto_atual != 'Intervalo' and minuto_atual != 'Acabou':
        minuto_atual = int(minuto_atual[0:2])    
        if (int(chutesnogolcasa) >= 3 and int(chutesnogolvisitante) <= 1) and (resultado_parcial == '1x0' or resultado_parcial == '0x0' or resultado_parcial == '2x1') and minuto_atual <= 60:
            print(f'CHANCE PARA BACK CASA NESSE JOGO')
            
            # toca um áudio
            pygame.mixer.music.play()
            time.sleep(3)

        elif (int(chutesnogolcasa) <= 1 and int(chutesnogolvisitante) >= 3) and (resultado_parcial == '0x1' or resultado_parcial == '0x0' or resultado_parcial == '1x2') and minuto_atual <= 60:
            print(f'CHANCE PARA BACK VISITANTE NESSE JOGO')
            
            # toca um áudio
            pygame.mixer.music.play()
            time.sleep(3)
            
    elif minuto_atual == 'Intervalo':   
        if (int(chutesnogolcasa) >= 3 and int(chutesnogolvisitante) <= 1) and (resultado_parcial == '1x0' or resultado_parcial == '0x0' or resultado_parcial == '2x1'):
            print(f'CHANCE PARA BACK CASA NESSE JOGO')
            
            # toca um áudio
            pygame.mixer.music.play()
            time.sleep(3)

        elif (int(chutesnogolcasa) <= 1 and int(chutesnogolvisitante) >= 3) and (resultado_parcial == '0x1' or resultado_parcial == '0x0' or resultado_parcial == '1x2'):
            print(f'CHANCE PARA BACK VISITANTE NESSE JOGO')
            
            # toca um áudio
            pygame.mixer.music.play()
            time.sleep(3)
            
            
    # MÉTODO AMBAS MARCAM
    if minuto_atual != 'Intervalo' and minuto_atual != 'Acabou':
        minuto_atual = int(minuto_atual[0:2])    
        if (int(chutesnogolcasa) >= 3) and (resultado_parcial == '0x1' or resultado_parcial == '0x2') and minuto_atual <= 50:
            print(f'CHANCE PARA AMBAS NESSE JOGO')
            
            # toca um áudio
            pygame.mixer.music.play()
            time.sleep(3)

        elif (int(chutesnogolvisitante) >= 3) and (resultado_parcial == '1x0' or resultado_parcial == '2x0') and minuto_atual <= 50:
            print(f'CHANCE PARA AMBAS NESSE JOGO')
            
            # toca um áudio
            pygame.mixer.music.play()
            time.sleep(3)
            
    elif minuto_atual == 'Intervalo':   
        if (int(chutesnogolcasa) >= 3) and (resultado_parcial == '0x1' or resultado_parcial == '0x2'):
            print(f'CHANCE PARA AMBAS NESSE JOGO')
            
            # toca um áudio
            pygame.mixer.music.play()
            time.sleep(3)

        elif (int(chutesnogolvisitante) >= 3) and (resultado_parcial == '1x0' or resultado_parcial == '2x0'):
            print(f'CHANCE PARA AMBAS NESSE JOGO')
            
            # toca um áudio
            pygame.mixer.music.play()
            time.sleep(3)
    
    

# início do script principal

os.environ['MOZ_FORCE_DISABLE_E10S'] = '1' # lembrar de colocar 1 depois do igual para este uso
url_inicio = 'https://www.sofascore.com'

# para dar toque de alerta quando chamada a função de toque
pygame.init()
pygame.mixer.music.load('toque.mpeg')

 # comentar a linha abaixo se for usar o navegador invisível e descomentar linhas no topo
driver = webdriver.Firefox() 
driver.get(url_inicio)
time.sleep(1)

driver.find_element_by_css_selector('.js-sport-games-live-count').click()
time.sleep(7)

flag = True
partida = ''

num_jogos, partida = abre_jogos() # função para abrir os jogos da rodada

if partida == False:
    print('Nenhum jogo para analisar.')
    driver.quit()

else:
    while flag == True:
        for i in range(num_jogos):
            try:
                scanner() # função para scanear os dados in live dos jogos
            except:
                continue


número de jogos ao vivo 46
abri o jogo 1
abri o jogo 2
abri o jogo 3
abri o jogo 4
abri o jogo 5
abri o jogo 6
abri o jogo 7
abri o jogo 8
exceção de estatistica do jogo 8
abri o jogo 9
exceção de estatistica do jogo 9
abri o jogo 10
exceção de estatistica do jogo 10
abri o jogo 11
exceção de estatistica do jogo 11
abri o jogo 12
exceção de estatistica do jogo 12
abri o jogo 13
abri o jogo 14
exceção de estatistica do jogo 14
abri o jogo 15
exceção de estatistica do jogo 15
abri o jogo 16
exceção de estatistica do jogo 16
abri o jogo 17
exceção de estatistica do jogo 17
abri o jogo 18
exceção de estatistica do jogo 18
abri o jogo 19
abri o jogo 20
exceção de estatistica do jogo 20
abri o jogo 21
exceção de estatistica do jogo 21
abri o jogo 22
exceção de estatistica do jogo 22
abri o jogo 23
exceção de estatistica do jogo 23
abri o jogo 24
exceção de estatistica do jogo 24
abri o jogo 25
exceção de estatistica do jogo 25
abri o jogo 26
exceção de estatistica do jogo 26
abri o jogo 27
e



Africa
CAF Champions League, Group B
Al Ahly - Al-Hilal Omdurman
Resultado Parcial 2x0
Minuto Intervalo
Chutes no gol do time da casa 4
Chutes no gol do time visitante 0


Africa
CAF Champions League, Group D
Espérance Tunis - JS Kabylie
Resultado Parcial 0x0
Minuto Intervalo
Chutes no gol do time da casa 3
Chutes no gol do time visitante 0
CHANCE PARA BACK CASA NESSE JOGO


Belgium
First Division A
Waasland Beveren - Royal Antwerp FC
Resultado Parcial 0x1
Minuto 24:47


Czech Republic
1. Liga
Slovan Liberec - Slavia Praha
Resultado Parcial 0x2
Minuto 63:14


Germany
3rd Liga
Chemnitzer FC - FSV Zwickau
Resultado Parcial 2x2
Minuto Acabou


Poland
Ekstraklasa
Górnik Zabrze - Wisła Kraków
Resultado Parcial 0x1
Minuto 24:43
Chutes no gol do time da casa 0
Chutes no gol do time visitante 2


Romania
Liga I
FCSB - Gaz Metan Mediaș
Resultado Parcial 1x0
Minuto 66:09
Chutes no gol do time da casa 5
Chutes no gol do time visitante 3


South Africa
Premier League
Supersport United - Highland



Germany
3rd Liga
Chemnitzer FC - FSV Zwickau
Resultado Parcial 2x2
Minuto Acabou


Poland
Ekstraklasa
Górnik Zabrze - Wisła Kraków
Resultado Parcial 0x1
Minuto 28:29
Chutes no gol do time da casa 0
Chutes no gol do time visitante 2


Romania
Liga I
FCSB - Gaz Metan Mediaș
Resultado Parcial 1x0
Minuto 70:07
Chutes no gol do time da casa 5
Chutes no gol do time visitante 3


South Africa
Premier League
Supersport United - Highlands Park
Resultado Parcial 2x0
Minuto Acabou
Chutes no gol do time da casa 7
Chutes no gol do time visitante 2


Switzerland
Challenge League
Schaffhausen - FC Aarau
Resultado Parcial 0x0
Minuto Intervalo
Chutes no gol do time da casa 1
Chutes no gol do time visitante 4
CHANCE PARA BACK VISITANTE NESSE JOGO


Germany
Bundesliga
Eintracht Frankfurt - Hertha BSC
Resultado Parcial 0x1
Minuto 30:26
Chutes no gol do time da casa 2
Chutes no gol do time visitante 1


France
Ligue 1
Lille OSC - Stade Brestois 29
Resultado Parcial 1x0
Minuto Acabou
Chutes no gol do time



Switzerland
Challenge League
Schaffhausen - FC Aarau
Resultado Parcial 0x0
Minuto Intervalo
Chutes no gol do time da casa 1
Chutes no gol do time visitante 4
CHANCE PARA BACK VISITANTE NESSE JOGO


Germany
Bundesliga
Eintracht Frankfurt - Hertha BSC
Resultado Parcial 0x1
Minuto 34:22
Chutes no gol do time da casa 2
Chutes no gol do time visitante 1


France
Ligue 1
Lille OSC - Stade Brestois 29
Resultado Parcial 1x0
Minuto Acabou
Chutes no gol do time da casa 10
Chutes no gol do time visitante 0


Netherlands
Eredivisie
Ajax - Willem II Tilburg
Resultado Parcial 0x1
Minuto Intervalo
Chutes no gol do time da casa 3
Chutes no gol do time visitante 3
CHANCE PARA AMBAS NESSE JOGO


Denmark
Superliga
Hobro - AC Horsens
Resultado Parcial 0x0
Minuto Acabou
Chutes no gol do time da casa 4
Chutes no gol do time visitante 3


Africa
CAF Champions League, Group B
Al Ahly - Al-Hilal Omdurman
Resultado Parcial 2x0
Minuto 47:05
Chutes no gol do time da casa 4
Chutes no gol do time visitante 0


Af